In [4]:
import pandas as pd
import os
import numpy as np

def preprocess(
        dietary_url='https://wwwn.cdc.gov/Nchs/Nhanes/2017-2018/P_DSQTOT.XPT', 
        nutrition_url='https://wwwn.cdc.gov/Nchs/Nhanes/2017-2018/P_DR1TOT.XPT', 
        dental_url='https://wwwn.cdc.gov/Nchs/Nhanes/2017-2018/P_OHXDEN.XPT'):
    '''
    @description: 
        Pre-process the Dietary Supplement, Nutrient Intakes, and Dental Health data
    @param: 
        URLs of the three data
    @return: 
        Pre-processed Pandas DataFrame
    '''
    # please be patient for data download
    dietary_path = './dataset/P_DR1TOT.XPT'
    nutrition_path = './dataset/P_DSQTOT.XPT'
    dental_path = './dataset/P_OHXDEN.XPT'

    df_dietary = pd.read_sas(dietary_path if os.path.exists(dietary_path) else dietary_url)
    df_nutrition = pd.read_sas(nutrition_path if os.path.exists(nutrition_path) else nutrition_url)
    df_dental = pd.read_sas(dental_path if os.path.exists(dental_path) else dental_url)
    print(len(df_nutrition))
    df_data = pd.merge(df_dietary, df_nutrition, on='SEQN')
    df_data = pd.merge(df_data, df_dental, on='SEQN')
    
    # fill the missing precise quantatative or special dietary data with 0
    attrs_set_0 = ['DR1SKY']
    for attr in df_data.columns:
        if attr.startswith('DSQ') or attr.startswith('DRQS') or attr.startswith('DRD'):
            attrs_set_0.append(attr)
    df_data[attrs_set_0] = df_data[attrs_set_0].fillna(0)

    # drop Coronal Caries and Sealants attributes from dental data
    attrs_keep = []
    for attr in df_data.columns:
        if not (attr.startswith('OHX') and (attr.endswith('CTC') or attr.endswith('CSC') 
                or attr.endswith('RTC') or attr.endswith('RSC') or attr.endswith('SE'))):
            attrs_keep.append(attr)
    df_data = df_data[attrs_keep]

    # fill missing "dental implant or not?" with "no"
    df_data.loc[:, 'OHXIMP'].fillna(2, inplace=True)
    # fill missing "how often add salt?" with "don't know"
    df_data.loc[:, 'DBD100'].fillna(9, inplace=True)

    # fill missing "Root Caries, Non-carious Lesion, Root Caries Restoration, Non-carious Lesion Restoration" with "not detected"
    df_data.update(df_data[['OHXRCAR', 'OHXRCARO', 'OHXRRES', 'OHXRRESO']].fillna(0))

    df_data = df_data.dropna()

    return df_data

final_data = preprocess()

/usr/local/lib/python3.7/dist-packages/pandas/io/sas/sas_xport.py:475: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[x] = v


14300


Total Nutritional columns = 169

In [5]:
nutritions = []
df_nutritions = pd.concat([final_data.loc[:,"SEQN"], final_data.loc[:,"DSQTIODI":"DRD370V"]], axis = 1)
df_nutritions.info()
# print(df_nutritions['DRABF']) 
print(df_nutritions.columns[1:10]) 
from google.colab import files
df_nutritions.to_csv('filename.csv') 
files.download('filename.csv')


<class 'pandas.core.frame.DataFrame'>
Int64Index: 11196 entries, 0 to 13770
Columns: 169 entries, SEQN to DRD370V
dtypes: float64(169)
memory usage: 14.5 MB
Index(['DSQTIODI', 'WTDRD1PP', 'WTDR2DPP', 'DR1DRSTZ', 'DR1EXMER', 'DRABF',
       'DRDINT', 'DR1DBIH', 'DR1DAY'],
      dtype='object')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [6]:
# columns which has all values zero
zero_columns = []
for h in df_nutritions.columns:
  if (df_nutritions[h] == 0).all():
     zero_columns.append(h)

print(zero_columns)

[]


In [ ]:
binary_df_columns = []
teritionary_df_columns = []
constant_df_columns = []
continuous_variables=[]
for h in df_nutritions.columns:
  d = set(df_nutritions[h])
  if(len(d)==2):
    binary_df_columns.append(h)
  elif(len(d)<=5 and len(d)>2):
    teritionary_df_columns.append(h)
  elif(len(d)==1):
    constant_df_columns.append(h)
  else:
    continuous_variables.append(h)

print(teritionary_df_columns)
print(len(teritionary_df_columns))
print(binary_df_columns)
print(len(binary_df_columns))
print(continuous_variables)
print(len(continuous_variables))
print(constant_df_columns)
print(len(constant_df_columns))

In [10]:
df_continuous_variables = df_nutritions[continuous_variables]
print(df_continuous_variables.corr())

              SEQN  DSQTIODI  WTDRD1PP  WTDR2DPP  DR1EXMER   DR1DBIH  \
SEQN      1.000000 -0.010717 -0.008671 -0.005479 -0.006885 -0.017866   
DSQTIODI -0.010717  1.000000  0.031351  0.033538 -0.002526  0.005043   
WTDRD1PP -0.008671  0.031351  1.000000  0.851316 -0.009085  0.013480   
WTDR2DPP -0.005479  0.033538  0.851316  1.000000 -0.013420  0.012748   
DR1EXMER -0.006885 -0.002526 -0.009085 -0.013420  1.000000 -0.022591   
...            ...       ...       ...       ...       ...       ...   
DRD370QQ  0.009271  0.013037  0.016810  0.013104 -0.002407 -0.004775   
DRD370RQ  0.006941  0.005110  0.001981  0.010649  0.001553 -0.007384   
DRD370SQ  0.000015  0.007721  0.029909  0.029119  0.022051 -0.001167   
DRD370TQ -0.010563  0.005063 -0.023035 -0.018998 -0.025110  0.004224   
DRD370UQ  0.012191  0.006881 -0.017724 -0.013687 -0.000019 -0.016019   

            DR1DAY   DR1LANG  DR1MRESP   DR1HELP  ...  DRD370KQ  DRD370LQ  \
SEQN      0.010836  0.009930 -0.001306  0.002598  ... -0.0

In [11]:
df_continuous_variables.describe()
#describe function provide the details of the correlated values

,SEQN,DSQTIODI,WTDRD1PP,WTDR2DPP,DR1EXMER,DR1DBIH,DR1DAY,DR1LANG,DR1MRESP,DR1HELP,...,DRD370KQ,DRD370LQ,DRD370MQ,DRD370NQ,DRD370OQ,DRD370QQ,DRD370RQ,DRD370SQ,DRD370TQ,DRD370UQ
count,11196.000000,11196.000000,11196.000000,1.119600e+04,11196.000000,11196.000000,11196.000000,11196.000000,11196.000000,11196.000000,...,11196.000000,11196.000000,11196.000000,11196.000000,11196.000000,11196.000000,11196.000000,11196.000000,11196.000000,11196.000000
mean,117149.814219,20.236194,26606.907687,2.682109e+04,72.837442,12.085834,4.547517,1.124330,1.272329,10.677653,...,0.033405,0.006431,0.568775,0.078510,0.020096,0.009468,0.035906,0.009736,0.329046,0.068328
std,4494.112384,143.309663,35513.322355,4.279000e+04,21.412610,12.931712,2.200283,0.467901,1.111259,3.319837,...,0.301182,0.172089,1.546367,0.678384,0.366103,0.221448,0.323222,0.138240,1.363461,0.585421
min,109263.000000,0.000000,805.552485,5.397605e-79,14.000000,-42.000000,1.000000,1.000000,1.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,113286.750000,0.000000,8109.894171,5.247106e+03,73.000000,3.000000,2.000000,1.000000,1.000000,12.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,117220.500000,0.000000,15065.105598,1.339989e+04,81.000000,9.000000,5.000000,1.000000,1.000000,12.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,121071.500000,0.000000,30541.012895,3.125570e+04,86.000000,19.000000,6.000000,1.000000,1.000000,12.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,124821.000000,12500.000000,538404.429708,7.029436e+05,94.000000,91.000000,7.000000,6.000000,77.000000,12.000000,...,12.000000,12.000000,25.000000,24.000000,30.000000,20.000000,10.000000,5.000000,30.000000,30.000000


In [12]:
df_corr = df_continuous_variables.corr()
np.fill_diagonal(df_corr.values, 0)
df_corr.style.highlight_max(color='yellow').highlight_min(color='blue')


,SEQN,DSQTIODI,WTDRD1PP,WTDR2DPP,DR1EXMER,DR1DBIH,DR1DAY,DR1LANG,DR1MRESP,DR1HELP,DR1TNUMF,DR1TKCAL,DR1TPROT,DR1TCARB,DR1TSUGR,DR1TFIBE,DR1TTFAT,DR1TSFAT,DR1TMFAT,DR1TPFAT,DR1TCHOL,DR1TATOC,DR1TATOA,DR1TRET,DR1TVARA,DR1TACAR,DR1TBCAR,DR1TCRYP,DR1TLYCO,DR1TLZ,DR1TVB1,DR1TVB2,DR1TNIAC,DR1TVB6,DR1TFOLA,DR1TFA,DR1TFF,DR1TFDFE,DR1TCHL,DR1TVB12,DR1TB12A,DR1TVC,DR1TVD,DR1TVK,DR1TCALC,DR1TPHOS,DR1TMAGN,DR1TIRON,DR1TZINC,DR1TCOPP,DR1TSODI,DR1TPOTA,DR1TSELE,DR1TCAFF,DR1TTHEO,DR1TALCO,DR1TMOIS,DR1TS040,DR1TS060,DR1TS080,DR1TS100,DR1TS120,DR1TS140,DR1TS160,DR1TS180,DR1TM161,DR1TM181,DR1TM201,DR1TM221,DR1TP182,DR1TP183,DR1TP184,DR1TP204,DR1TP205,DR1TP225,DR1TP226,DR1_320Z,DR1_330Z,DR1BWATZ,DRD350AQ,DRD350BQ,DRD350CQ,DRD350DQ,DRD350EQ,DRD350FQ,DRD350GQ,DRD350HQ,DRD350IQ,DRD370AQ,DRD370BQ,DRD370CQ,DRD370DQ,DRD370EQ,DRD370FQ,DRD370GQ,DRD370HQ,DRD370IQ,DRD370KQ,DRD370LQ,DRD370MQ,DRD370NQ,DRD370OQ,DRD370QQ,DRD370RQ,DRD370SQ,DRD370TQ,DRD370UQ
SEQN,0.000000,-0.010717,-0.008671,-0.005479,-0.006885,-0.017866,0.010836,0.009930,-0.001306,0.002598,-0.010735,-0.012180,-0.002621,-0.015686,-0.010504,-0.019688,-0.009047,-0.002947,-0.014601,-0.009970,0.012001,-0.018595,-0.009654,0.004119,0.000586,-0.010657,-0.003988,0.010546,-0.014146,0.009139,-0.012421,-0.011728,-0.009908,-0.007988,-0.004879,-0.005331,-0.002168,-0.005223,0.000937,0.005418,-0.002445,0.002143,-0.001898,0.004588,-0.002110,-0.011929,-0.016677,-0.009636,-0.000258,-0.009814,-0.014603,-0.016530,0.003609,-0.001802,-0.019581,-0.001772,-0.007525,0.010447,0.012492,-0.000518,0.008145,-0.001264,0.009266,-0.006086,-0.004649,-0.003233,-0.014828,-0.014588,-0.007447,-0.010146,-0.006997,-0.002355,-0.001377,0.008471,0.005706,-0.000941,-0.002779,-0.005491,0.002041,0.009804,0.008733,0.010232,-0.000366,-0.011805,-0.015524,0.007601,-0.006634,-0.004827,0.019305,0.009968,-0.000303,-0.006290,-0.007704,0.005248,-0.003659,-0.008695,0.007580,-0.003038,-0.003340,0.001121,-0.004931,-0.016038,0.009271,0.006941,0.000015,-0.010563,0.012191
DSQTIODI,-0.010717,0.000000,0.031351,0.033538,-0.002526,0.005043,-0.002097,-0.013252,-0.017335,0.024581,0.077243,0.012889,0.013286,0.013062,-0.002851,0.039528,0.009847,-0.002051,0.014221,0.018836,-0.004323,0.026669,0.009739,0.000249,0.013338,0.010151,0.022958,0.000511,0.001135,0.029247,0.011540,0.010834,0.006671,0.011653,0.040851,0.003336,0.066073,0.029290,0.011443,-0.005470,-0.002821,0.008273,-0.001872,0.031694,0.004462,0.012078,0.039478,0.018961,0.006868,0.034900,0.015081,0.028313,0.005322,0.031766,0.002938,0.002071,0.047234,-0.002410,-0.001424,0.000422,-0.001936,-0.000794,-0.005975,-0.001809,-0.002940,-0.009483,0.015166,0.011945,0.010897,0.018010,0.026458,-0.002427,-0.009978,0.004113,-0.002164,0.003792,0.039497,0.046510,0.002368,0.004917,-0.000506,0.000230,0.011269,0.005253,0.000275,0.001941,0.005288,0.002336,0.002518,0.016465,-0.002967,0.008747,0.010916,0.008370,0.012699,0.002779,0.004516,0.004129,-0.000177,0.020843,0.013539,0.003986,0.013037,0.005110,0.007721,0.005063,0.006881
WTDRD1PP,-0.008671,0.031351,0.000000,0.851316,-0.009085,0.013480,-0.171798,-0.058949,-0.070053,0.095170,0.091523,0.038664,0.070022,0.000972,-0.004996,0.055307,0.043902,0.049003,0.044105,0.024683,0.030901,0.059452,0.060516,0.035196,0.057949,0.038245,0.051317,0.002536,0.026178,0.043260,0.048502,0.099227,0.075497,0.077390,0.051172,0.016200,0.069226,0.040956,0.054907,0.029251,0.048230,-0.005796,-0.004694,0.047528,0.053122,0.063623,0.097862,0.047695,0.050915,0.056430,0.044471,0.078872,0.045034,0.152320,0.033998,0.052752,0.139687,0.076998,0.067372,0.029534,0.060365,0.007830,0.058828,0.037543,0.053911,0.034782,0.043097,0.012363,0.007778,0.022804,0.038131,-0.000815,0.038502,-0.003981,0.010352,0.000257,0.079059,0.091751,0.006079,0.034421,0.014947,-0.004286,0.043788,0.007746,0.000077,0.057410,0.011164,0.016036,-0.002964,0.065761,-0.006374,-0.053654,0.036875,0.007760,0.059747,-0.016606,-0.014932,0.028701,-0.007469,0.030659,-0.010189,0.000313,0.016810,0.001981,0.029909,-0.023035,-0.017724
WTDR2DPP,-0.005479,0.033538,0.85

In [13]:
#Sort Correlation Matrix in Python
sorted_mat = df_corr.unstack().sort_values()
print(sorted_mat.iloc[::-1])

DR1TM181  DR1TMFAT    0.998924
DR1TMFAT  DR1TM181    0.998924
DR1TPFAT  DR1TP182    0.997589
DR1TP182  DR1TPFAT    0.997589
DR1TFOLA  DR1TFDFE    0.979111
                        ...   
WTDRD1PP  DR1DAY     -0.171798
DR1BWATZ  DR1_330Z   -0.237833
DR1_330Z  DR1BWATZ   -0.237833
DR1MRESP  DR1HELP    -0.277409
DR1HELP   DR1MRESP   -0.277409
Length: 11449, dtype: float64


In [14]:
# Nan is interpreted as no correlation between the two variables
# zero correlation implies no linear relationship at all

zero correlation implies no linear relationship at all